In [1]:
#importing required libraries
import cv2
from string import ascii_uppercase
import tkinter as tk
from PIL import Image, ImageTk
import pickle
import mediapipe as mp
import numpy as np
import random

# Main Application :

class Application:

    def __init__(self):
        
        #initalizing the webcam
        self.vs = cv2.VideoCapture(0)
        
        self.letter=[]
        self.current_image = None
        
        #loading the model and ohter saved file 
        self.word_list=pickle.load(open('D:/my python/python code/speech/word_list.pickle','rb'))
        self.sent_list=pickle.load(open('D:/my python/python code/speech/sent_list.pickle','rb'))
        
        self.data_dict=pickle.load(open('D:/my python/python code/new/data.pickle','rb'))
        self.labels=np.asarray(self.data_dict['labels'])
        self.labels_dict={}
        for label in self.labels:
            self.labels_dict[label]=label
            

        self.model_dict = pickle.load(open('D:/my python/python code/new/classifier.p', 'rb'))
        self.loaded_model = self.model_dict['classifier']
        print("Loaded model from disk") 
        
        #intializing the GUI interface for interaction
        self.root = tk.Tk()
        self.root.title("Sign Language To Text Conversion")
        self.root.protocol('WM_DELETE_WINDOW', self.destructor)
        self.root.geometry("900x800")

        self.panel = tk.Label(self.root)
        self.panel.place(x=80, y=40, width=550, height=380)

        self.T = tk.Label(self.root)
        self.T.place(x=40, y=2)
        self.T.config(text="Sign Language To Text Conversion",font=("Courier", 20, "bold"))

        self.panel4 = tk.Label(self.root)  # Word
        self.panel4.place(x=140, y=420)

        self.T2 = tk.Label(self.root)
        self.T2.place(x=15, y=420)
        self.T2.config(text="Word :", font=("Courier", 20, "bold"))

        self.panel5 = tk.Label(self.root)  # Sentence
        self.panel5.place(x=185, y=480)

        self.T3 = tk.Label(self.root)
        self.T3.place(x=15, y=480)
        self.T3.config(text="Sentence :", font=("Courier", 20, "bold"))

        self.T4 = tk.Label(self.root)
        self.T4.place(x=165, y=535)
        self.T4.config(text="Suggestions :", fg="black",font=("Courier", 20, "bold"))

        self.bt1 = tk.Button(self.root, command=self.action, height=0, width=0)
        self.bt1.place(x=35, y=580)

        self.bt2 = tk.Button(self.root, command=self.action, height=0, width=0)
        self.bt2.place(x=335, y=580)

        self.bt3 = tk.Button(self.root, command=self.action, height=0, width=0)
        self.bt3.place(x=635, y=580)
    
        self.current_symbol = "Empty"
        self.photo = "Empty"
        self.video_loop()

    def video_loop(self):           #function for continuous tracking of hands using webcam
        mp_hands = mp.solutions.hands
        mp_drawing = mp.solutions.drawing_utils
        mp_drawing_styles = mp.solutions.drawing_styles
        hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)
        
        data_aux = []
        x_ = []
        y_ = []
        
        #reding every frame
        ret, frame = self.vs.read()
        if ret:
            H, W, _ = frame.shape
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)
            
            #check if no hand gesture is made
            if not results.multi_hand_landmarks:  
                self.current_image = Image.fromarray(frame_rgb)
                imgtk = ImageTk.PhotoImage(image=self.current_image)
                self.panel.imgtk = imgtk
                self.panel.config(image=imgtk)
                value=0
                self.predict_letter(value,frame,0,0)
            
            #hand gesture is made    
            else:                                   
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        frame,  # image to draw
                        hand_landmarks,  # model output
                        mp_hands.HAND_CONNECTIONS,  # hand connections
                        mp_drawing_styles.get_default_hand_landmarks_style(),
                        mp_drawing_styles.get_default_hand_connections_style()
                        )

                for hand_landmarks in results.multi_hand_landmarks:
                    for i in range(len(hand_landmarks.landmark)):
                        x = hand_landmarks.landmark[i].x
                        y = hand_landmarks.landmark[i].y
                        x_.append(x)
                        y_.append(y)

                    for i in range(len(hand_landmarks.landmark)):
                        x = hand_landmarks.landmark[i].x
                        y = hand_landmarks.landmark[i].y
                        #getting the coordinates for prediction
                        data_aux.append(x - min(x_))
                        data_aux.append(y - min(y_))

                #getting the position of hand in the screen and adjusting as per need
                x1 = int(min(x_) * W) - 10
                y1 = int(min(y_) * H) - 10
                x2 = int(max(x_) * W) - 10
                y2 = int(max(y_) * H) - 10
                
                #putting a rectangle on hand to make it highlight 
                frame=cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
                #calling predict function
                self.predict_letter(data_aux,frame,x1,y1) 
                 
        self.root.after(5, self.video_loop)

    #function to predict the symbol
    def predict_letter(self,val,frame,x1,y1):
       
        if val == 0  :
            #if no hand gesture then symbol is blank
            self.current_symbol= " "
            if cv2.waitKey(0) == ord('q'):
               self.destructor() 
        
        else:
            #prediction of symbol
            result = self.loaded_model.predict([np.asarray(val)])
            self.current_symbol = self.labels_dict[(result[0])]
        
            #putting suggested symbol to the GUI screen
            frame_rgb=cv2.putText(frame, self.current_symbol, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3,
                cv2.LINE_AA)
            
            #processing frame matchup the color combination
            frame_rgb = cv2.cvtColor(frame_rgb, cv2.COLOR_BGR2RGB)
            self.current_image = Image.fromarray(frame_rgb)
            imgtk = ImageTk.PhotoImage(image=self.current_image)
            #setting the pannel
            self.panel.imgtk = imgtk
            self.panel.config(image=imgtk)
            
        self.suggest_word() #calling function to suggest words or sentence
        return
    
    #function to suggest words and sentences  
    def suggest_word(self):
        #synmbol is blank
        if self.current_symbol == " ":
            self.panel4.config(text=" ", font=("Courier", 30))
            self.panel5.config(text=" ", font=("Courier", 30))
            self.bt1.config(text=" ", font=("Courier", 20))
            self.bt2.config(text=" ", font=("Courier", 20))
            self.bt3.config(text=" ", font=("Courier", 20))
            
        elif self.current_symbol in self.labels_dict:
            #getting words and sentences from pre-saved starting with current symbol
            
            #list to load sentence that starts with current symbol
            word=[word for word in self.word_list if word.startswith(self.current_symbol)]
            #list to load sentence that starts with current symbol
            sent=[sent for sent in self.sent_list if sent.startswith(self.current_symbol.lower())]
            
            if len(sent)== 0:
                #intializing sentence empty string to avoid errors
                sent=['']
            
            if len(word)==0:
                #intializing random _words with empty string to avoid errors
                random_words=['','','']
            
            elif len(word)==1:
                #intializing random _words with word and empty string to avoid errors
                random_words=word+ ['', '']

            elif len(word)==2:
                #intializing random _words with word and empty string to avoid errors
                random_words=word+ ['']
            
            else:
                #picking random indices from list of words
                random_indices = random.sample(range(len(word)), 3)
                random_words = [word[index] for index in random_indices]
            
            #setting the GUI Pannel with suggested words and sentences 
            self.panel4.config(text=random_words[0], font=("Courier", 20))
            self.panel5.config(text=sent[0],font=("Courier", 20))
            self.bt1.config(text=random_words[0], font=("Courier", 20))
            self.bt2.config(text=random_words[1], font=("Courier", 20))
            self.bt3.config(text=random_words[2], font=("Courier", 20))
        return
    
    #dummy function for buttons                  
    def action(self):
        pass
    
    #function to close GUI window and release webcam feom use
    def destructor(self):
        print("Closing Application...")
        self.root.destroy()
        self.vs.release()
        cv2.destroyAllWindows() 
        
print("Starting Application...")


#to intialize and start the application
(Application()).root.mainloop()

Starting Application...
Loaded model from disk
Closing Application...


: 

: 